In [1]:
# run SNPsnap
############################
# data_dir = "/Charles/project/ASm6A/ASm6A/homer_anno/split_region/"
data_dir = "/Charles/project/ASm6A/ASm6A/common_ASm6A/totaltissues/homer_anno_100%/split_region/"
os.chdir(data_dir)

result_dir = os.path.abspath("../SNPsnap/")
os.system("mkdir -p %s" % result_dir)
shell = "%s/SNPsnap_run.sh" % result_dir
#######################################################################################
with open(shell, 'w') as fw:
    txt_list = glob.glob("*/*.txt")
    for txt in txt_list:
        prefix = txt.split("/")[-2]
        sub_dir = os.path.join(result_dir, prefix)
        os.system("mkdir -p %s" % sub_dir)
        txt = os.path.abspath(txt)
        region = os.path.basename(txt).split("_")[0]
        com = "Rscript SNPsnap.R --input_file %s --dir_out %s --num_batches 1 --idx_batch 1 --population EAS \
--num_matching_snps 100 --genomic_region %s" % (txt, sub_dir, region)
        fw.write("%s\n" % com)       
## ln -s /home/galaxy/project/alleleSpecific_analysis/results/ASm6A_addZr/common_ASm6A/homer/split_region/modified_SNPsnap/SNPsnap.R ./

In [2]:
###
base_dir = "/Charles/project/ASm6A/ASm6A/common_ASm6A/totaltissues/homer_anno_100%/SNPsnap/"
os.chdir(base_dir)
dir_list = [x for x in glob.glob("*") if os.path.isdir(x)]
print(len(dir_list))
for subdir in dir_list:
    os.chdir("%s/%s/"% (base_dir, subdir))
    os.system("cat matched_SNPs/*/SNPmatch_100_matchedSNPs_batch1.txt > combined_SNPsnap.txt")

1


In [3]:
def process_each_column(value_list, result_file):
    value_list = [x for x in value_list if ":" in str(x)]
    with open(result_file, 'w') as fw:
        for x in value_list:
            chro = "chr" + x.split(":")[0]
            end = int(x.split(":")[1])
            start = end - 1
            line = "%s\t%s\t%s" % (chro, str(start), str(end))
            fw.write(line.strip() + "\n")
            
# os.chdir("/Charles/project/ASm6A/ASm6A/homer_anno/SNPsnap/")
os.chdir("/Charles/project/ASm6A/ASm6A/common_ASm6A/totaltissues/homer_anno_100%/SNPsnap/")
dir_list = [x for x in glob.glob("*") if os.path.isdir(x)]
for subdir in dir_list:
    in_file = "%s/combined_SNPsnap.txt" % subdir
    result_dir = "%s/SNPsnap/" % subdir
    os.system("mkdir -p %s" % result_dir)
# in_file = "/Charles/project/ASm6A/ASm6A/common_ASm6A/homer_anno/split_region/common_ASm6A/SNPsnap/matched_SNPs/commonASm6A_SNPsnap.txt"
# result_dir = "/Charles/project/ASm6A/ASm6A/common_ASm6A/homer_anno/split_region/common_ASm6A/SNPsnap/"
# in_file = "/Charles/project/ASm6A/ASm6A/common_ASm6A/totaltissues/homer_anno/split_region/common/SNPsnap/matched_SNPs/commonASm6A_SNPsnap.txt"
# result_dir = "/Charles/project/ASm6A/ASm6A/common_ASm6A/totaltissues/homer_anno/split_region/common/SNPsnap/"
    if not os.path.getsize(in_file):
        print(in_file)
    else:
        df = pd.read_table(in_file, header=None).dropna()
        process_each_column(df.iloc[:,0].tolist(), "%s/Input_SNP.bed"%result_dir)
        for i in range(2, 102):
            process_each_column(df.iloc[:,i].tolist(), "%s/Set_%d.bed" % (result_dir,(i-1)))